# 1. Know your metrics 


Knowing the metrics is the first step to make a business decision. After knowing what kind of business problem you are solving, what metrics to choose will be the key to measure  the sucess of the following statistical analysis. It is like the North Star in the business project to guide all our actions.


In this notebook, I will use an online retail dataset for showing what kine of metrics we should use in the real-life business analysis.

Let's look at the dataset.

In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('OnlineRetail.csv', encoding= 'unicode_escape')

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


We can see from the dataset, each row represents one item in each transaction. 

The first metric I want to use is the Revenue.

In [10]:
# revenue = quantity * unitprice 
df['Revenue'] = df['Quantity'] * df['UnitPrice']

# convert the type of invoice date from string to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# get the fields of Year and month
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month

df['YearMonth'] = df['Year']*100 + df['Month']

In [14]:
# Let's look at the revenue by month
revenue = df.groupby('YearMonth')['Revenue'].sum().reset_index()
revenue

,YearMonth,Revenue
0,201012,748957.020
1,201101,560000.260
2,201102,498062.650
3,201103,683267.080
4,201104,493207.121
5,201105,723333.510
6,201106,691123.120
7,201107,681300.111
8,201108,682680.510
9,201109,1019687.622


In [46]:
# Let's look at the data in UK
df_uk = df[df['Country']=='United Kingdom']

Another metric is New Customer Ratio:

We will find the first purchase date of each customer and define new customers based on that.

In [47]:
# Create a dataframe about the first purchase date of each customer
tx_min_purchase = df_uk.groupby('CustomerID')['InvoiceDate'].min().reset_index()
tx_min_purchase.columns = ['CustomerID', 'MinPurchaseDate']
tx_min_purchase['MinPurchaseYearMonth'] = tx_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

# merge the first purchase dataset with the original dataset
df_uk = pd.merge(df_uk, tx_min_purchase, how='left',on='CustomerID')

# create a column called user type and assign existing
# if the invoice date is later than the min Purchase date
df_uk['UserType'] = 'New'
df_uk.loc[df_uk['YearMonth'] > df_uk['MinPurchaseYearMonth'], 'UserType'] = 'Existing'

# calculate the revenue by month and user type
tx_user_type_revenue = df_uk.groupby(['YearMonth','UserType'])['Revenue'].sum().reset_index()
tx_user_type_revenue

,YearMonth,UserType,Revenue
0,201012,New,676742.620
1,201101,Existing,195275.510
2,201101,New,239032.790
3,201102,Existing,220994.630
4,201102,New,187253.280
5,201103,Existing,296350.030
6,201103,New,263357.360
7,201104,Existing,268226.660
8,201104,New,174027.381
9,201105,Existing,434725.860


In [51]:
tx_user_ratio = df_uk.query("UserType == 'New'").groupby(['YearMonth'])['CustomerID'].nunique()/df_uk.query("UserType == 'Existing'").groupby(['YearMonth'])['CustomerID'].nunique()
tx_user_ratio = tx_user_ratio.reset_index()
tx_user_ratio = tx_user_ratio.dropna()
tx_user_ratio

,YearMonth,CustomerID
1,201101,1.124224
2,201102,0.904000
3,201103,0.792233
4,201104,0.510166
5,201105,0.343793
6,201106,0.281250
7,201107,0.236589
8,201108,0.192572
9,201109,0.304878
10,201110,0.328852


Let's look at another main metrics: Monthly retention rate

Monthly Retention Rate = retained customer # from prev month / total customer # this month

In [22]:
# Main metrics : Monthly Retention Rate
df_uk_monthly = df_uk.groupby(['CustomerID','YearMonth'])['Revenue'].sum().reset_index()

In [28]:
tx_retention = pd.crosstab(df_uk_monthly['CustomerID'],df_uk_monthly['YearMonth'])
tx_retention

YearMonth,201012,201101,201102,201103,201104,201105,201106,201107,201108,201109,201110,201111,201112
CustomerID,,,,,,,,,,,,,
12346.0,0,1,0,0,0,0,0,0,0,0,0,0,0
12747.0,1,1,0,1,0,1,1,0,1,0,1,1,1
12748.0,1,1,1,1,1,1,1,1,1,1,1,1,1
12749.0,0,0,0,0,0,1,0,0,1,0,0,1,1
12820.0,0,1,0,0,0,0,0,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0,0,0,1,0,0,0,0,0,0,0,0,0
18281.0,0,0,0,0,0,0,1,0,0,0,0,0,0
18282.0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [34]:
#create an array of dictionary which keeps Retained & Total User count for each month
months = tx_retention.columns[2:]
retention_array = []
for i in range(len(months)-1):
    retention_data = {}
    selected_month = months[i+1]
    prev_month = months[i]
    retention_data['InvoiceYearMonth'] = int(selected_month)
    retention_data['TotalUserCount'] = tx_retention[selected_month].sum()
    retention_data['RetainedUserCount'] = tx_retention[(tx_retention[selected_month]>0) & (tx_retention[prev_month]>0)][selected_month].sum()
    retention_array.append(retention_data)

In [38]:
#convert the array to dataframe and calculate Retention Rate
tx_retention = pd.DataFrame(retention_array)
tx_retention['RetentionRate'] = tx_retention['RetainedUserCount']/tx_retention['TotalUserCount']
tx_retention

,InvoiceYearMonth,TotalUserCount,RetainedUserCount,RetentionRate
0,201103,923,305,0.330444
1,201104,817,310,0.379437
2,201105,985,369,0.374619
3,201106,943,417,0.442206
4,201107,899,379,0.421580
5,201108,867,391,0.450980
6,201109,1177,417,0.354291
7,201110,1285,502,0.390661
8,201111,1548,616,0.397933
9,201112,617,402,0.651540


The next metric that we are looking at is cohort based retention rate, which is the retention rate for each cohort. We will see what percentage of customer retained after the first purchase in each month. This view will help us to see how the recent and old cohorts differ in terms of the retention rate and see if the recent changes of products affect the new customers' retention or not.

In [93]:
# Cohort retention rate
tx_retention = pd.crosstab(df_uk_monthly['CustomerID'], df_uk_monthly['YearMonth']).reset_index()
tx_retention = pd.merge(tx_retention,tx_min_purchase[['CustomerID','MinPurchaseYearMonth']],on='CustomerID')
new_column_names = [ 'm_' + str(column) for column in tx_retention.columns[:-1]]
new_column_names.append('MinPurchaseYearMonth')
tx_retention.columns = new_column_names


In [94]:
#create the array of Retained users for each cohort monthly
retention_array = []
for i in range(len(months)):
    retention_data = {}
    selected_month = months[i]
    prev_months = months[:i]
    next_months = months[i+1:]
    for prev_month in prev_months:
        retention_data[prev_month] = np.nan

    total_user_count = tx_retention[tx_retention.MinPurchaseYearMonth ==  selected_month].MinPurchaseYearMonth.count()
    retention_data['TotalUserCount'] = total_user_count
    retention_data[selected_month] = 1 

    query = "MinPurchaseYearMonth == {}".format(selected_month)

    for next_month in next_months:
        new_query = query + " and {} > 0".format(str('m_' + str(next_month)))
        retention_data[next_month] = np.round(tx_retention.query(new_query)['m_' + str(next_month)].sum()/total_user_count,2)
    retention_array.append(retention_data)
    
tx_retention = pd.DataFrame(retention_array)
tx_retention.index = months

In [95]:
tx_retention

,TotalUserCount,201102,201103,201104,201105,201106,201107,201108,201109,201110,201111,201112
YearMonth,,,,,,,,,,,,
201102,339,1.0,0.25,0.19,0.28,0.28,0.25,0.26,0.28,0.28,0.31,0.10
201103,408,NaN,1.00,0.19,0.26,0.22,0.23,0.17,0.26,0.24,0.29,0.09
201104,276,NaN,NaN,1.00,0.22,0.22,0.22,0.21,0.23,0.23,0.26,0.08
201105,252,NaN,NaN,NaN,1.00,0.23,0.17,0.17,0.21,0.24,0.27,0.10
201106,207,NaN,NaN,NaN,NaN,1.00,0.21,0.16,0.25,0.26,0.32,0.10
201107,172,NaN,NaN,NaN,NaN,NaN,1.00,0.20,0.19,0.23,0.28,0.11
201108,140,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.26,0.23,0.26,0.14
201109,275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.29,0.33,0.12
201110,318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,0.27,0.13
